## Websocket Client 

In [9]:
%gui asyncio

from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display
import asyncio
import os
import websockets
from re import search

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
    widget.on_click(getvalue)
    return future


class WebsocketClient:
    """
    Websocket client which can send and receive messages to any connected user.
    """
    
    def __init__(self, server_ip, server_port=8080):
        """
        Init of Websocket-Client
        server_ip (str): Either and IPv4 address or a name which can get resolved to an IP address.
        server_port (int): default port is 8080
        """
        self.server_ip = server_ip
        self.server_port = server_port
        self.user_name = os.path.expandvars("$USER")
        self.chat_input = widgets.Text(placeholder='Send your message like this --> receiver:message',description='Chat Input:',layout=Layout(width='80%'))
        self.chat_area = widgets.Textarea(value = '', disabled=True, description='Chat:', layout=Layout(width='99%', height='280px'))
        self.chat_button = widgets.Button(description='Send!',layout=Layout(width='19%'))
    
    async def connection(self):
        """
        Connects the client to the server
        """
        self.websocket = await websockets.connect(f"ws://{self.server_ip}:{self.server_port}")
        self.chat_area.value += "Connection was successful for user at " + f"ws://{self.server_ip}:{self.server_port}\n"
            
        # The first message is the user name
        await self.websocket.send(self.user_name)
        self.chat_area.value += f"{self.user_name} added to the chat. To exit the chat type 'exit'.\n"
        asyncio.create_task(self.send_message())
        asyncio.create_task(self.receive_message())
        display(VBox([HBox([self.chat_input, self.chat_button]), self.chat_area]))

    async def send_message(self):
        """
        Get input data from console input.
        Sends input-message to the server via self.websocket and displays it into the chat-area.
        """
        while True:
            await wait_for_change(self.chat_button)
            self.chat_area.value += f"To {self.chat_input.value}\n"
            
            if search('exit', self.chat_input.value):
                await self.websocket.close()
                self.chat_area.value += "Connection was successfully closed."
            else:
                await self.websocket.send(self.chat_input.value)
                self.chat_input.value = ""

    async def receive_message(self):
        """
        Receives messages from server via self.websocket and displays it into the chat-area
        """
        while True:
            response = await self.websocket.recv()
            self.chat_area.value += f"From {response}\n"

In [10]:
client = WebsocketClient(server_ip='kvanc.exercise')
await client.connection() 